In [1]:
#import os, sys
#sys.path.insert(0, os.path.abspath('../src/models'))
import os
os.chdir('../src/models')

In [99]:
import simulate_pandemic as simp
from actions import costs, city_restrictions

In [8]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import networkx as nx

In [114]:
import plotly.express as px
import plotly.graph_objects as go
import datetime

set3 = px.colors.qualitative.Set3

color_map_set3 = {
    'Lockdown':          set3[3],
    'Hard Quarantine':    set3[11],
    'Light Quarantine':   set3[1],
    'Social Distancing':  'rgb(204, 245, 175)',
    'Unrestricted':        set3[6]     
}

def make_SIR_graph(data):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)
    
    fig = go.Figure()
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(d_counts[0]))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/55e3, name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/55e3, name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/55e3, name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/55e3, name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/55e3, name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map):
    fig = go.Figure()
   
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(data)+2)

    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'royalblue',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'firebrick',
                            line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2)))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)-1],
                y1=1,
                fillcolor=a,
                opacity=0.5,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False},
                      yaxis = {'showgrid': False},
                      showlegend=True, title = title, hovermode="x")

    fig.show()

In [104]:
def vacination(pop_matrix, nodes):
    for n in nodes:
        pop_matrix[pop_matrix[:,0] == n, 1]= -1
    return pop_matrix

In [105]:
def top_k_nodes(graph, pop_matrix, centrality, k=10):
    susceptible = set(pop_matrix[pop_matrix[:,1] == 0][:,0].astype(int))
    centrality = {k:v for k,v in centrality.items() if k in susceptible}
    ordered_nodes = sorted(centrality, key=centrality.get, reverse=True)
    return np.array(ordered_nodes)[:k]

In [121]:
pop_matrix = simp.init_infection()
data = []
actions = []
days = 364
step_size = 7

restrictions = city_restrictions['Unrestricted']
centrality = nx.degree_centrality(simp.G)
#centrality = {k: np.random.random() for k in simp.G.nodes}

for day in tqdm(range(1, days+1)):
    #if less than 20% still susceptible, break simulation
    if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break


    # Choose a new policy at each week
    #if day % step_size == 1:          
    #    action = np.random.choice(list(policies.keys()))
    #    actions.append(action)
    #    restrictions = policies[action]
    
    nodes = top_k_nodes(simp.G, pop_matrix, centrality, 0)#int(1e-3*len(simp.G)))
    vacination(pop_matrix, nodes)
    pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
    pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
    pop_matrix = simp.update_population(pop_matrix)

    data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:,1]) 

100%|████████████████████████████████████████████████████████████████████████████████| 364/364 [00:52<00:00,  6.93it/s]


In [123]:
df = make_SIR_graph(data)
actions_3_colors = list(map(color_map_set3.get,  actions))
make_beds_graph(df, actions_3_colors, 7, 'Sim', color_map_set3)